# Unsupervised Learning

## Tomotopy HDP (Hierarchical Dirichlet Process)

In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords

import sys
import os

from pprint import pprint

import pyLDAvis
import pyLDAvis.gensim_models
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

import tqdm

import tomotopy as tp

from preprocess import preprocess, tokenizer, stopword_list

In [2]:
data_root = '../data/cvt/'
df = pd.read_csv(os.path.join(data_root, 'covidvaccine.csv'))
df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,Huge Thanks And Best Wishes With @SerumInstInd...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,Any update of #CovidVaccine ????,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,From a private frontline COVID nurses group to...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,So you want the whole world to take a #covidva...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,Watch to learn about the Phase 3 clinical tria...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,* Continue with routine preventive measures (s...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,COVID: California Suspends System Of Shared Va...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,Today I got my first dose of the #COVIDVaccine...,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,Here's Where to Receive a COVID Vaccine in Ill...,illinois covidvaccine,Buffer,2


In [3]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

df

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,hashtag_count
0,Ashwin,India 🇮🇳,I'm Just A Man On a Mission For My Vision!\n\n...,2018-01-08 06:01:00,89.0,3108.0,6780.0,0,2020-08-09 19:15:00,huge thanks best wishes seruminstindia adarpoo...,covisheild,Twitter for Android,1
1,K. Kartik Rao,Mumbai and Nagpur India,Actor 🎭 \n\nhttps://t.co/u7GZfoWBDt\n\nSubscri...,2016-12-10 16:06:00,225.0,257.0,25362.0,0,2020-08-09 20:02:00,update covidvaccine,covidvaccine,Twitter for Android,1
2,Mac Patrick (Road Hater) Henry,German Bundesrepublik,Liberty lover. \nServing the people of the 🇺🇸....,2014-01-11 19:47:00,338.0,277.0,28305.0,0,2020-08-09 21:21:00,private frontline covid nurses group today cov...,covid19 covidvaccinerace covidvaccine pandemic,Twitter Web App,4
3,lydia Bell,Earth,"Mummy, animal lover , energy worker , awake in...",2009-03-23 01:09:00,2038.0,1105.0,63422.0,0,2020-08-09 21:29:00,want whole world take covidvaccine s fast trac...,covidvaccine,Twitter for iPhone,1
4,Lou Ann Donovan,"Kansas, USA",Live Young Look Young BE Young! Certified Inte...,2009-03-17 05:42:00,4365.0,4581.0,2505.0,0,2020-08-09 22:55:00,watch learn phase clinical trial covid vaccine...,covid19 covidvaccine covidclinicaltrial,Twitter for iPhone,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154533,Script Your Future '21,DMV,HowardU and Script Your Future has joined forc...,2021-01-31 18:40:41,15.0,42.0,4.0,0,2021-02-28 23:55:25,continue routine preventive measures vaccinati...,NaN,Twitter Web App,0
154534,eastvillagetwt,"East Village, New York, NY","Politics, the environment, science, culture, t...",2011-08-27 19:24:17,1968.0,5001.0,136110.0,0,2021-02-28 23:56:37,covid california suspends system shared vaccin...,covid19 covidvaccine,Twitter for iPhone,2
154535,Yogita,"New York, NY",Primum non nocere 💕👩🏽‍⚕️ | Tweets are my own. ...,2015-08-07 01:48:57,17.0,58.0,1194.0,0,2021-02-28 23:57:00,today got first dose covidvaccine super thankful,covidvaccine,Twitter for Android,1
154536,DrZ,USA,NaN,2016-03-23 13:00:36,54.0,4.0,4.0,0,2021-02-28 23:57:06,heres receive covid vaccine illinois illinois ...,illinois covidvaccine,Buffer,2


In [4]:
df['text']

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0         huge thanks best wishes seruminstindia adarpoo...
1                                       update covidvaccine
2         private frontline covid nurses group today cov...
3         want whole world take covidvaccine s fast trac...
4         watch learn phase clinical trial covid vaccine...
                                ...                        
154533    continue routine preventive measures vaccinati...
154534    covid california suspends system shared vaccin...
154535     today got first dose covidvaccine super thankful
154536    heres receive covid vaccine illinois illinois ...
154537    utah slc sold walmart vaccine first dose book ...
Name: text, Length: 154538, dtype: object

In [5]:
# Prepare corpus
def get_tp_corpus(docs):
    porter_stemmer = nltk.PorterStemmer().stem
    english_stops = set(porter_stemmer(w) for w in stopwords.words('english'))
    tp_corpus = tp.utils.Corpus(
        tokenizer=tp.utils.SimpleTokenizer(porter_stemmer), 
        stopwords=lambda x: x in english_stops
    )
    tp_corpus.process(d.lower() for d in docs)
    return tp_corpus

tp_corpus = get_tp_corpus(list(df['text']))

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def compute_coherence_values_tphdp(tp_corpus, a, b):
    hdp_model = tp.HDPModel(alpha=a,eta=b,corpus=tp_corpus)
    hdp_model.train(500)
    co_mass = tp.coherence.Coherence(hdp_model,coherence="u_mass")
    co_cv = tp.coherence.Coherence(hdp_model,coherence="c_v")
    return co_mass.get_score(), co_cv.get_score(), hdp_model.live_k

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
already_run = False

# Alpha parameter (length = 5)
alpha = list(np.arange(0.01, 1, 0.1))

# Beta parameter (length = 5)
beta = list(np.arange(0.01, 1, 0.1))

model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence_cv': [],
                 "Coherence_umass":[],
                }

if not already_run:
    # Can take a long time to run
    pbar = tqdm.tqdm(total=100)
    
    # Iterate through alpha values
    for a in alpha:
        # Iterate through beta values
        for b in beta:
            # Get the coherence score for the given parameters
            umass, cv, live_k = compute_coherence_values_tphdp(tp_corpus=tp_corpus, a=a, b=b)
            #print(umass,cv)
            # Save the model results
            model_results['Topics'].append(live_k)
            model_results['Alpha'].append(a)
            model_results['Beta'].append(b)
            model_results['Coherence_cv'].append(cv)
            model_results['Coherence_umass'].append(umass)

            pbar.update(1)
    
    coherence_results = pd.DataFrame(model_results)
    coherence_results.to_csv('../out/hdp_tuning_results.csv', index=False)
    pbar.close()
else:
    coherence_results = pd.read_csv('../out/hdp_tuning_results.csv')

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

  0%|          | 0/81 [01:52<?, ?it/s]

100%|██████████| 81/81 [51:16<00:00, 48.18s/it]
82it [51:53, 44.86s/it]                        
83it [52:26, 41.28s/it]
84it [52:54, 37.40s/it]
85it [53:22, 34.67s/it]
86it [53:50, 32.54s/it]
87it [54:18, 31.17s/it]
88it [54:46, 30.23s/it]
89it [55:13, 29.15s/it]
90it [55:40, 28.68s/it]
91it [57:12, 47.69s/it]
92it [57:50, 44.59s/it]
93it [58:19, 40.06s/it]
94it [58:47, 36.45s/it]
95it [59:14, 33.44s/it]
96it [59:42, 31.88s/it]
97it [1:00:09, 30.60s/it]
98it [1:00:38, 29.86s/it]
99it [1:01:06, 29.31s/it]
100it [1:01:33, 36.94s/it]


In [10]:
# Sort coherence_results
sorted_coherence_results = coherence_results.sort_values(by=['Coherence_cv', 'Coherence_umass'], ascending=False)
sorted_coherence_results

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topics,Alpha,Beta,Coherence_cv,Coherence_umass
90,74,0.91,0.01,0.923667,-14.477180
50,83,0.51,0.01,0.923072,-12.628739
10,97,0.11,0.01,0.922872,-12.900285
0,103,0.01,0.01,0.921702,-12.009948
30,87,0.31,0.01,0.919730,-11.550542
...,...,...,...,...,...
97,4,0.91,0.71,0.777193,-13.403380
18,3,0.11,0.81,0.771288,-15.699109
96,3,0.91,0.61,0.770603,-15.945484
85,3,0.81,0.51,0.765013,-12.878254


In [11]:
# View the top coherence result
best_hyperparams = sorted_coherence_results.iloc[0]
best_hyperparams

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topics             74.000000
Alpha               0.910000
Beta                0.010000
Coherence_cv        0.923667
Coherence_umass   -14.477180
Name: 90, dtype: float64

In [26]:
mdl = tp.HDPModel(
    corpus=tp_corpus,
    alpha=best_hyperparams['Alpha'],
    eta=best_hyperparams['Beta']
)

mdl.train(500)

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
mdl.summary()

mdl_converted, topic_ids = mdl.convert_to_lda()

topic_term_dists = np.stack([mdl_converted.get_topic_word_dist(k) for k in range(mdl_converted.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl_converted.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl_converted.docs])
vocab = list(mdl_converted.used_vocabs)
term_frequency = mdl_converted.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency
)

pyLDAvis.save_html(prepared_data, '../out/hdpvis.html')

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<Basic Info>
| HDPModel (current version: 0.11.1)
| 154535 docs, 1531044 words
| Total Vocabs: 76786, Used Vocabs: 76786
| Entropy of words: -7.72703
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 500, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -7.89354
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| initial_k: 2 (the initial number of topics between 2 ~ 32767 The number of topics will be adjusted for data during training)
| alpha: 0.91 (concentration coeficient of Dirichlet Process for document-table )
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| gamma: 0.1 (concentration coeficient of Dirichlet Process for table-topic)
| seed: 3877812910 (random seed)
| trained in version 0.11.1
|
<Parameters>
| alpha (concentration coeficient of Dirichlet Process for document-table)
|  0

In [29]:
rows = {}

for i in range(mdl_converted.k):
    topic_words = mdl_converted.get_topic_words(i)
    rows['Topic #' + str(i)] = [w for w, p in topic_words]

topic_words = pd.DataFrame.from_dict(rows, orient='index',
                       columns=['Keyword #1', 'Keyword #2', 'Keyword #3', 'Keyword #4', 'Keyword #5',
                               'Keyword #6', 'Keyword #7', 'Keyword #8', 'Keyword #9', 'Keyword #10'])
topic_words

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Keyword #1,Keyword #2,Keyword #3,Keyword #4,Keyword #5,Keyword #6,Keyword #7,Keyword #8,Keyword #9,Keyword #10
Topic #0,covidvaccin,vaccin,covid,get,peopl,dose,first,today,need,nt
Topic #1,covidvaccin,vaccin,covid,get,dose,peopl,first,today,got,nt
Topic #2,covidvaccin,vaccin,dose,got,first,today,covid,get,feel,second
Topic #3,vaccin,covid,covidvaccin,trial,coronaviru,pfizer,effect,approv,first,dose
Topic #4,covidvaccin,vaccin,covid,nh,thank,receiv,work,today,staff,get
...,...,...,...,...,...,...,...,...,...,...
Topic #69,commit,lwestaf,drtsion,justinesherri,emilyemorehous,emergjennci,gumbo_amando,fortun,sanfrancisco,candicecodymd
Topic #70,fromthearch,therapist,tension,nonleagu,chinnorfc,chinnor,fcraynerslan,raynerslanefc,head,simmer
Topic #71,bigtuna,fabiosi,bettyanncanizio,meltinginmarana,dapool,enzoenoqu,nlt_,julyb_lucil,qtanon,jasper
Topic #72,govmurphi,govwhitm,nm,louisianagov,stevesisolak,michel,govjanetmil,governorbullock,govtimwalz,mayorscau


In [30]:
token_counts = mdl_converted.get_count_by_topics()
topic_words.insert(0, 'Token Counts', token_counts)
topic_words

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Token Counts,Keyword #1,Keyword #2,Keyword #3,Keyword #4,Keyword #5,Keyword #6,Keyword #7,Keyword #8,Keyword #9,Keyword #10
Topic #0,628728,covidvaccin,vaccin,covid,get,peopl,dose,first,today,need,nt
Topic #1,612497,covidvaccin,vaccin,covid,get,dose,peopl,first,today,got,nt
Topic #2,65286,covidvaccin,vaccin,dose,got,first,today,covid,get,feel,second
Topic #3,53889,vaccin,covid,covidvaccin,trial,coronaviru,pfizer,effect,approv,first,dose
Topic #4,40179,covidvaccin,vaccin,covid,nh,thank,receiv,work,today,staff,get
...,...,...,...,...,...,...,...,...,...,...,...
Topic #69,15,commit,lwestaf,drtsion,justinesherri,emilyemorehous,emergjennci,gumbo_amando,fortun,sanfrancisco,candicecodymd
Topic #70,15,fromthearch,therapist,tension,nonleagu,chinnorfc,chinnor,fcraynerslan,raynerslanefc,head,simmer
Topic #71,9,bigtuna,fabiosi,bettyanncanizio,meltinginmarana,dapool,enzoenoqu,nlt_,julyb_lucil,qtanon,jasper
Topic #72,9,govmurphi,govwhitm,nm,louisianagov,stevesisolak,michel,govjanetmil,governorbullock,govtimwalz,mayorscau


In [31]:
topic_words.to_csv('../out/hdp_topic_words.csv', index=True)

/home/andrewchen/anaconda3/envs/ml_proj/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
